# New

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score

# Step 1: Load the data
df = pd.read_csv("disaster_relief_dataset.csv")

# Step 2: Define input and output
X = df.drop(columns=['food_required_tons', 'rescue_teams_required', 'medical_aid_required'])
y = df[['food_required_tons', 'rescue_teams_required', 'medical_aid_required']]

# Step 3: Define categorical features for encoding
categorical_features = ['disaster_type', 'location', 'severity_level', 'weather_condition']

# Step 4: Create preprocessing transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Step 5: Define the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=5,
        random_state=42,
        n_jobs=-1
    )))
])

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Train the model
model.fit(X_train, y_train)

# Step 8: Make predictions
y_pred = model.predict(X_test)

# Step 9: Evaluate the model
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
r2 = r2_score(y_test, y_pred, multioutput='raw_values')

# Print metrics
for i, col in enumerate(y.columns):
    print(f"\n--- {col} ---")
    print(f"Mean Absolute Error: {mae[i]:.2f}")
    print(f"R² Score: {r2[i]:.4f}")


--- food_required_tons ---
Mean Absolute Error: 12.58
R² Score: -0.0047

--- rescue_teams_required ---
Mean Absolute Error: 7.27
R² Score: -0.0049

--- medical_aid_required ---
Mean Absolute Error: 249.21
R² Score: -0.0049


In [18]:
sample_input = pd.DataFrame([{
    'disaster_type': 'earthquake',
    'location': 'Zone B',
    'population_affected': 50000,
    'severity_level': 'extreme',
    'infrastructure_damage_percent': 90,
    'weather_condition': 'stormy',
    'road_blocked': 1,
    'medical_cases': 1500
}])
prediction = model.predict(sample_input)
print(prediction)

[[ 25.41244378  15.22722732 517.87859686]]


# Pickel

In [22]:
import pickle

# After training the model (Step 7):
# Step 10: Pickle the trained model
with open('disaster_relief_model.pkl', 'wb') as f:
    pickle.dump(model, f)